In [ ]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import openai

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

openai.api_key = "введите свой токен для теста"

user_data = {}

def start(update: Update, context: CallbackContext):
    user_id = update.message.from_user.id
    if user_id not in user_data:
        user_data[user_id] = {'name': None, 'experience': None, 'goal': None, 'additional_info': None}
    update.message.reply_text("Привет! Давай начнем. Как тебя зовут?")

def collect_info(update: Update, context: CallbackContext):
    user_id = update.message.from_user.id
    text = update.message.text

    if user_id not in user_data:
        user_data[user_id] = {'name': None, 'experience': None, 'goal': None, 'additional_info': None}
    if user_data[user_id]['name'] is None:
        user_data[user_id]['name'] = text
        update.message.reply_text("Опиши свой опыт катания на сноуборде.")
    elif user_data[user_id]['experience'] is None:
        user_data[user_id]['experience'] = text
        update.message.reply_text("Какова твоя цель использования сноуборда? (например, фрирайд, фристайл, карвинг)")
    elif user_data[user_id]['goal'] is None:
        user_data[user_id]['goal'] = text
        update.message.reply_text("Есть ли что-то еще, что поможет мне сделать лучший выбор для тебя?")
    elif user_data[user_id]['additional_info'] is None:
        user_data[user_id]['additional_info'] = text
        get_openai_response(update, user_id)

def get_openai_response(update: Update, user_id: int):
    messages = [
        {"role": "system", "content": "The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly."},
        {"role": "user", "content": f"Меня зовут {user_data[user_id]['name']}."},
        {"role": "user", "content": f"У меня {user_data[user_id]['experience']} опыта катания на сноуборде."},
        {"role": "user", "content": f"Я ищу сноуборд для {user_data[user_id]['goal']}."},
        {"role": "user", "content": f"Дополнительная информация: {user_data[user_id]['additional_info']}."}
    ]

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        response_text = (
            "1. Собранные данные о вас:\n"
            f"Имя: {user_data[user_id]['name']}\n"
            f"Опыт катания на сноуборде: {user_data[user_id]['experience']}\n"
            f"Цель использования сноуборда: {user_data[user_id]['goal']}\n"
            f"Дополнительная информация: {user_data[user_id]['additional_info']}\n\n"
            "2. Рекомендация:\n"
            f"{response.choices[0].message['content'].strip()}"
        )
        update.message.reply_text(response_text)
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        update.message.reply_text(f"Произошла ошибка при получении рекомендации: {e}")

    del user_data[user_id]

def main():
    TELEGRAM_TOKEN = "введите свой токен для теста"
    updater = Updater(TELEGRAM_TOKEN, use_context=True)
    dp = updater.dispatcher

    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, collect_info))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()


2024-02-22 11:56:38,390 - apscheduler.scheduler - INFO - Scheduler started
